In [1]:
import numpy as np 
import pandas as pd

In [2]:
BASE_PATH = r"D:\DATA_ANALYST\FULL_STACK_FROJECT\Stock Market Prediction\stock_market_unclean_dataset"
df = pd.read_csv(BASE_PATH + r"\company_data\company_fundamentals.csv")


In [3]:
df.head()

,company,sector,pe_ratio,debt_equity,roe
0,HDFC,Finance,14.39,2.89,11.50
1,HDFCBANK,Finance,15.54,3.95,16.42
2,ICICIBANK,Finance,11.42,2.38,16.47
3,SBIN,Finance,22.21,2.28,15.53
4,AXISBANK,Finance,19.78,3.40,13.70


In [4]:
df.duplicated()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
dtype: bool

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   company      50 non-null     object 
 1   sector       50 non-null     object 
 2   pe_ratio     50 non-null     float64
 3   debt_equity  50 non-null     float64
 4   roe          50 non-null     float64
dtypes: float64(3), object(2)
memory usage: 2.1+ KB


In [6]:
df.rename(columns={
    'company': 'company_name',
    'sector': 'business_sector',
    'pe_ratio': 'price_earnings_ratio',
    'debt_equity': 'debt_to_equity',
    'roe': 'return_on_equity'
}, inplace=True)

In [7]:
df['company_name'] = df['company_name'].str.strip().str.upper()

In [8]:
df['business_sector'] = df['business_sector'].str.strip().str.title()

In [9]:
df.head()

,company_name,business_sector,price_earnings_ratio,debt_to_equity,return_on_equity
0,HDFC,Finance,14.39,2.89,11.50
1,HDFCBANK,Finance,15.54,3.95,16.42
2,ICICIBANK,Finance,11.42,2.38,16.47
3,SBIN,Finance,22.21,2.28,15.53
4,AXISBANK,Finance,19.78,3.40,13.70


In [10]:
df = df[df['price_earnings_ratio'] > 0]

In [11]:
df = df[df['return_on_equity'] >= 0]

In [12]:
df = df[(df['price_earnings_ratio'] < 100) & (df['debt_to_equity'] < 10)]

In [13]:
df['risk_level'] = df['debt_to_equity'].apply(lambda x: 'High Risk' if x > 2 else 'Low Risk')

In [14]:
df['performance'] = df['return_on_equity'].apply(lambda x: 'Good' if x >= 15 else 'Average')

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.head()

,company_name,business_sector,price_earnings_ratio,debt_to_equity,return_on_equity,risk_level,performance
0,HDFC,Finance,14.39,2.89,11.50,High Risk,Average
1,HDFCBANK,Finance,15.54,3.95,16.42,High Risk,Good
2,ICICIBANK,Finance,11.42,2.38,16.47,High Risk,Good
3,SBIN,Finance,22.21,2.28,15.53,High Risk,Good
4,AXISBANK,Finance,19.78,3.40,13.70,High Risk,Average


In [17]:
print(df.columns)

Index(['company_name', 'business_sector', 'price_earnings_ratio',
       'debt_to_equity', 'return_on_equity', 'risk_level', 'performance'],
      dtype='object')


In [18]:
df.isna().sum()

company_name            0
business_sector         0
price_earnings_ratio    0
debt_to_equity          0
return_on_equity        0
risk_level              0
performance             0
dtype: int64

In [19]:
df.describe()

,price_earnings_ratio,debt_to_equity,return_on_equity
count,50.000000,50.000000,50.000000
mean,20.760000,1.560200,16.010600
std,8.802539,1.046899,5.228207
min,6.910000,0.220000,8.100000
25%,14.407500,0.705000,12.285000
50%,20.220000,1.165000,15.340000
75%,25.960000,2.242500,18.262500
max,41.520000,3.950000,29.550000


In [20]:
def outlier_bounds(col):
    Q1, Q3 = col.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    return Q1 - 1.5*IQR, Q3 + 1.5*IQR

for col in ['price_earnings_ratio','debt_to_equity','return_on_equity']:
    lb, ub = outlier_bounds(df[col])
    print(col, "→ Outliers:", df[(df[col] < lb) | (df[col] > ub)].shape[0])


price_earnings_ratio → Outliers: 0
debt_to_equity → Outliers: 0
return_on_equity → Outliers: 2


In [21]:
df['valuation_type'] = pd.cut(
    df['price_earnings_ratio'],
    bins=[0, 15, 25, 100],
    labels=['Undervalued', 'Fair', 'Overvalued']
)

df['leverage_risk'] = np.where(df['debt_to_equity'] > 3, 'High', 'Low')

df['profitability_flag'] = np.where(df['return_on_equity'] > 20, 1, 0)


In [22]:
df['investment_grade'] = np.where(
    (df['return_on_equity'] > 18) &
    (df['debt_to_equity'] < 2.5),
    'Good',
    'Risky')

In [23]:
df.head()

,company_name,business_sector,price_earnings_ratio,debt_to_equity,return_on_equity,risk_level,performance,valuation_type,leverage_risk,profitability_flag,investment_grade
0,HDFC,Finance,14.39,2.89,11.50,High Risk,Average,Undervalued,Low,0,Risky
1,HDFCBANK,Finance,15.54,3.95,16.42,High Risk,Good,Fair,High,0,Risky
2,ICICIBANK,Finance,11.42,2.38,16.47,High Risk,Good,Undervalued,Low,0,Risky
3,SBIN,Finance,22.21,2.28,15.53,High Risk,Good,Fair,Low,0,Risky
4,AXISBANK,Finance,19.78,3.40,13.70,High Risk,Average,Fair,High,0,Risky


In [24]:
df = df.drop(columns=['risk_level'])

In [25]:
df.head()

,company_name,business_sector,price_earnings_ratio,debt_to_equity,return_on_equity,performance,valuation_type,leverage_risk,profitability_flag,investment_grade
0,HDFC,Finance,14.39,2.89,11.50,Average,Undervalued,Low,0,Risky
1,HDFCBANK,Finance,15.54,3.95,16.42,Good,Fair,High,0,Risky
2,ICICIBANK,Finance,11.42,2.38,16.47,Good,Undervalued,Low,0,Risky
3,SBIN,Finance,22.21,2.28,15.53,Good,Fair,Low,0,Risky
4,AXISBANK,Finance,19.78,3.40,13.70,Average,Fair,High,0,Risky


In [26]:
df['investment_grade'].value_counts()

investment_grade
Risky    37
Good     13
Name: count, dtype: int64

In [27]:
df['investment_grade1'] = np.where((df['return_on_equity'] >= 15) &(df['debt_to_equity'] <= 3),'Good','Risky')

In [28]:
df['investment_grade1'].value_counts()

investment_grade1
Risky    25
Good     25
Name: count, dtype: int64

In [29]:
df = df.drop(columns=['investment_grade'])

In [30]:
df['profitability_flag'].value_counts()

profitability_flag
0    41
1     9
Name: count, dtype: int64

In [31]:
df.head()

,company_name,business_sector,price_earnings_ratio,debt_to_equity,return_on_equity,performance,valuation_type,leverage_risk,profitability_flag,investment_grade1
0,HDFC,Finance,14.39,2.89,11.50,Average,Undervalued,Low,0,Risky
1,HDFCBANK,Finance,15.54,3.95,16.42,Good,Fair,High,0,Risky
2,ICICIBANK,Finance,11.42,2.38,16.47,Good,Undervalued,Low,0,Good
3,SBIN,Finance,22.21,2.28,15.53,Good,Fair,Low,0,Good
4,AXISBANK,Finance,19.78,3.40,13.70,Average,Fair,High,0,Risky


In [32]:
df['business_sector'].value_counts()

business_sector
Finance    10
Energy     10
It         10
Pharma     10
Metal       5
Auto        5
Name: count, dtype: int64

In [33]:
df['performance'].value_counts()

performance
Good       27
Average    23
Name: count, dtype: int64

In [34]:
df['valuation_type'].value_counts()

valuation_type
Fair           19
Undervalued    16
Overvalued     15
Name: count, dtype: int64

In [35]:
df['leverage_risk'].value_counts()

leverage_risk
Low     43
High     7
Name: count, dtype: int64

In [36]:
df['investment_grade1'].value_counts()

investment_grade1
Risky    25
Good     25
Name: count, dtype: int64

In [37]:
df.rename(columns={'investment_grade1': 'investment_grade'}, inplace=True)

In [38]:
df.head()

,company_name,business_sector,price_earnings_ratio,debt_to_equity,return_on_equity,performance,valuation_type,leverage_risk,profitability_flag,investment_grade
0,HDFC,Finance,14.39,2.89,11.50,Average,Undervalued,Low,0,Risky
1,HDFCBANK,Finance,15.54,3.95,16.42,Good,Fair,High,0,Risky
2,ICICIBANK,Finance,11.42,2.38,16.47,Good,Undervalued,Low,0,Good
3,SBIN,Finance,22.21,2.28,15.53,Good,Fair,Low,0,Good
4,AXISBANK,Finance,19.78,3.40,13.70,Average,Fair,High,0,Risky


In [39]:
df.to_csv("company_fundamentals.csv", index=False)